In [1]:
from collections import Counter

import spacy
from tqdm.notebook import tqdm

nlp = spacy.load("nl_core_news_sm")
import pandas as pd

In [2]:
from wordfreq import word_frequency, top_n_list

In [3]:
s = 'Het beeldhouwwerk is gemaakt in opdracht van Johan Maurits voor de toenmalige overtuin van het Mauritshuis in Den Haag en bevond zich aan het einde van de zichtas van deze tuin.'

In [4]:
def process_text(text):
    return [token.lemma_ for token in nlp(text) if not (token.is_punct or token.is_space)]


top_list = top_n_list('nl', 20000)
top_list_string = ' '.join(top_list)
freqwords = process_text(top_list_string)  #  if not (token.is_stop or token.is_punct or token.is_space)
samplewords = process_text(sample_text)

basic_word_count = 0
for word in tqdm(samplewords, total=(len(samplewords))):
    if word in freqwords:
        basic_word_count += 1
    else:
        print(word)
basic_word_count

# words
# for i, word in enumerate(top_list):
#     if word != words[i]:
#         print(f'{words[i]} - {word}')


NameError: name 'sample_text' is not defined

In [ ]:

sample_text = '''
Ja goedenavond. Het is vandaag 1 september. En zoals u ongetwijfeld nog weet: de laatste formele datum op onze routekaart van coronamaatregelen. Dat is de directe aanleiding voor deze persconferentie. Ik zeg maar meteen: we hebben geen groot of onverwacht nieuws vandaag. Toch denken we dat het goed is de stand van zaken met u te bespreken, want er is natuurlijk na de laatste persconferentie van twee weken geleden wel het nodige gebeurd.
We willen vandaag twee vragen met u doornemen. Ten eerste de duiding: waar staan we nu, ongeveer een half jaar na het begin van de uitbraak? Hoe beoordelen we de recente cijfers? Daar zal ik kort iets over zeggen. Ten tweede, wat doen we nu en de komende tijd om die tweede golf voor te blijven en er toch op voorbereid te zijn? Daar zal Hugo de Jonge zo meteen op ingaan.
Maar voordat ik de cijfers induik, wil ik om te beginnen een keer duidelijk gezegd hebben dat uiteraard iedereen vragen mag stellen bij wat we als kabinet doen en daar ook kritiek op mag hebben. Sterker nog: het zou gek zijn als dat niet zou gebeuren. Het is volkomen logisch dat mensen na een half jaar van beperkingen zelf op onderzoek uitgaan. Of dat er mensen zijn die vanuit hun eigen discipline iets vinden over wat virologen en medici ons adviseren en wat wij als kabinet besluiten. Dat brengt ons verder en daar staan we dus open voor. Ik zeg er wel eerlijk bij: we kunnen bij onze beslissingen niet om de cijfers heen en we zullen ook heel precies moeten blijven kijken hoe het virus zich gedraagt. Maar dat wil niet zeggen dat er maar één waarheid is over de beste manier om met de gevolgen van corona om te gaan. Daarom praten we ook buiten het OMT met deskundigen. En daar zal Hugo de Jonge zo nog meer over zeggen. Ik had vanmiddag op het Catshuis een vierde gesprek met jongeren, dit keer samen met Eric Wiebes en Kajsa Ollongren. Dat helpt ons echt. En dus gaan Hugo de Jonge en ik volgende week een online sessie organiseren om van zoveel mogelijk mensen goede ideeën en kritische vragen te horen.
Die manier van werken past bij wat we steeds hebben gezegd. We werken op basis van voortschrijdend inzicht. Niet met 100% van de kennis, maar wel met steeds meer kennis. We leren per dag, per week, per maand en we stellen het beleid daarop bij. Zo namen we in maart bijvoorbeeld op gezag van de Wereldgezondheidsorganisatie aan dat mensen gemiddeld 2 weken op de IC zouden liggen. Dat bleken er bijna 3 te zijn. Een ander voorbeeld: 14 dagen quarantaine bleek gaandeweg naar 10 dagen te kunnen. Of denk aan de routekaart zelf, die in het voorjaar steeds door ons eigen goede gedrag werd ingehaald, waardoor we voor de zomer ineens grote stappen konden zetten. Of aan twee weken geleden, toen juist bleek dat we weer even op de rem moesten trappen.
We zagen toen de besmettingscijfers nog flink oplopen, vooral in de privésfeer. En dus hebben we gezegd: ontvang nou niet meer dan 6 mensen thuis, registreer je gasten in de horeca en werk ook na 1 september zoveel mogelijk thuis. Nu zien we dat de besmettingscijfers weer stabiel zijn, of zelfs licht dalen. We zitten nu op een plateau van rond de 500 besmettingen per dag en het aantal ziekenhuisopnamen en ic-patiënten is ook stabiel op een niveau dat de zorg aankan. Dat beroemde R-getal dat zegt hoeveel andere mensen iedere besmette persoon aansteekt, zat vandaag voor het eerst sinds een tijdje weer net onder de 1.
'''
print(len(process_text(sample_text)))
Counter(samplewords)

In [ ]:
top_list[-1]
print(word_frequency('decoraties', 'nl'))
print(word_frequency(top_list[-100], 'nl'))
word_frequency('aansteken', 'nl')




----------------------------------



In [ ]:
def _preprocess_conference_data(conference_data: list) -> tuple:
    """
    Preprocesses the text of a single conference

    :return: a tuple containing Rutte texts and De Jonge texts respectively per press conference
    """

    # Starts with saving text for Rutte
    save_text = 'rutte'
    words = set()

    # Only keep the sentences of Rutte and De Jonge
    text_rutte, text_de_jonge = [], []
    for line in conference_data:
        if save_text == 'rutte':
            text_rutte.append(line)
        if save_text == 'de jonge':
            text_de_jonge.append(line)

        if line.isupper():
            if 'RUT' in line:
                save_text = 'rutte'
            elif 'DE JONGE' in line:
                save_text = 'de jonge'
            else:
                words.add(line)
                save_text = 'other'

    # get rid of newline characters
    speakers_text = [text_rutte, text_de_jonge]

    for i, data in enumerate(speakers_text):
        speakers_text[i] = ''.join(data).replace("\n", " ")

    return tuple(speakers_text)

In [ ]:
def _calculate_tfidf(text_by_speaker, create_csv) -> tuple:
    """

    Calculates the tfidf per speaker per conference

    :return: a tuple containing Rutte texts and De Jonge texts respectively

    """
    corpus = pd.DataFrame(columns=['words'])
    nr_of_conferences = len(text_by_speaker[0])

    # Create Dataframe with Word Counts 
    for i in tqdm(range(nr_of_conferences)):
        full_conference_text = text_by_speaker[0][i]['text'] + text_by_speaker[1][i]['text']
        words = [token.lemma_ for token in nlp(full_conference_text) if
                 not (token.is_stop or token.is_punct or token.is_space)]

        word_count = Counter(words)

        new_words = list(set(word_count.keys()) - set(corpus['words']))
        corpus = corpus.append(pd.DataFrame({'words': new_words}), ignore_index=True)

        wordlist = []
        for word in corpus['words']:
            if word in word_count.keys():
                wordlist.append(word_count[word])
            else:
                wordlist.append(0)

        corpus[text_by_speaker[0][i]['date']] = wordlist

    corpus.set_index('words', inplace=True)
    corpus.fillna(0, inplace=True)

    if create_csv:
        corpus.to_csv('corpus.csv')

    tf_idf = {k: [] for k in corpus.columns}

    # Create Dataframe with Relative Word Frequencies 
    for index, row in tqdm(corpus.iterrows(), total=len(corpus)):
        docs_with = np.count_nonzero(row)

        for colname, count in row.items():
            total_uniques = np.count_nonzero(corpus[colname])
            value = (count / total_uniques) * math.log(len(corpus.columns) / docs_with)

            tf_idf[colname].append(value)

    tf_idf_df = pd.DataFrame.from_dict(tf_idf)
    tf_idf_df.set_index(corpus.index, inplace=True)

    if create_csv:
        tf_idf_df.to_csv(('tf_idf.csv'))

    return (corpus, tf_idf_df)

    '''
            total_uniques = len(word_list)
            
            # Get Word Frequency
            word_frequency = list()
            for word_tuple in word_list:
                word, count = word_tuple
                
                docs_with = 0
                # Check number of other text docs that contain the word
                for i, conferences_list in enumerate(text_by_speaker):
                    for j, conference in enumerate(conferences_list):
                        if word in dict(conference['word_list']):
                            docs_with += 1
                        
                count = count / total_uniques * math.log(nr_of_docs / docs_with)
                word_frequency.append((word, count))
    '''

    """
    # works better than the Spacy sentence splitter
    nr_of_docs = len(text_by_speaker[0]) + len(text_by_speaker[1])
    
    # Get Word Counts                 
    for i, conferences_list in enumerate(text_by_speaker):
        for j, conference in enumerate(conferences_list):
            words = [token.lemma_ for token in nlp(conference['text']) if not (token.is_stop or token.is_punct or token.is_space)]
    
            word_count = Counter(words)

            # Sorted word count                 
            word_list = list(word_count.items())
            word_list.sort(key=lambda item: item[1], reverse=True)

            text_by_speaker[i][j]['word_list'] = word_list
                     
    # Get Relative Word Frequencies                 
    for i, conferences_list in enumerate(text_by_speaker):
        for j, conference in enumerate(conferences_list):       
            
            word_list = text_by_speaker[i][j]['word_list']
                     
            # length of word_list
            total_uniques = len(word_list)
            
            # Get Word Frequency
            word_frequency = list()
            for word_tuple in word_list:
                word, count = word_tuple
                
                docs_with = 0
                # Check number of other text docs that contain the word
                for i, conferences_list in enumerate(text_by_speaker):
                    for j, conference in enumerate(conferences_list):
                        if word in dict(conference['word_list']):
                            docs_with += 1
                        
                count = count / total_uniques * math.log(nr_of_docs / docs_with)
                word_frequency.append((word, count))
            text_by_speaker[i][j]['word_frequency'] = word_frequency
               
    return text_by_speaker
    """

In [ ]:
CONFERENCE_OUTPUT_FOLDER = '../input/conferences/'


def _preprocess_all_conferences(tfidf=False, save_tfidf=False) -> tuple:
    """

    Returns the preprocessed conference data

    :return: a tuple containing Rutte texts and De Jonge texts respectively

    """
    conference_paths = os.listdir(CONFERENCE_OUTPUT_FOLDER)
    all_text_rutte, all_text_de_jonge = [], []

    for conf_file_name in conference_paths:
        with open(f"{CONFERENCE_OUTPUT_FOLDER}/{conf_file_name}", "r", encoding='utf-8') as f:
            text_rutte, text_de_jonge = _preprocess_conference_data(f.readlines())
            conf_date = conf_file_name.replace('.txt', '')
            all_text_rutte.append({'date': conf_date, 'text': text_rutte})
            all_text_de_jonge.append({'date': conf_date, 'text': text_de_jonge})

    ### Disabled for now, should be an option run option (e.g. add boolean as parameter to this function?)
    if False:
        all_text_rutte, all_text_de_jonge = _get_sentence_length((all_text_rutte, all_text_de_jonge))

    if tfidf:
        all_text_rutte, all_text_de_jonge = _calculate_tfidf((all_text_rutte, all_text_de_jonge), save_tfidf=save_tfidf)

    return all_text_rutte, all_text_de_jonge

In [ ]:
texts = _preprocess_all_conferences()

In [ ]:
corpus, tf_idf = _calculate_tfidf(texts, create_csv=True)

In [ ]:
from tqdm.notebook import tqdm

tf_idf
df = pd.DataFrame.from_dict(tf_idf)
df.set_index(corpus.index[:5])

for i in tqdm(tf_idf):
    print('yes!')

In [ ]:
corpus.head(100)

In [ ]:
test = dict.fromkeys(corpus.columns, [])
print(test)
test['2020-09-01'].append(1)
print(test)

In [ ]:
test2 = {'2020-09-01': [], '2020-09-18': [], '2020-09-28': [], '2020-10-13': []}
print(test2)
test2['2020-09-01'].append(1)
print(test2)

In [ ]:
test3 = {k: [] for k in corpus.columns}
print(test3)
test3['2020-09-01'].append(1)
print(test3)

In [ ]:
print(tf_idf.keys())
tf_idf['2020-09-18'] += [3]
tf_idf['2020-09-18'].append(3)
tf_idf['2020-10-13']

In [ ]:
# Sorted word count
#word_list = list(word_count.items())
#word_list.sort(key=lambda item: item[1], reverse=True)
#words = [x[0] for x in word_list]
#words_dict = {key:value for (key,value) in dictonary.items()}

#temp_list = []
#for word, count in word_list:
#    if word not in corpus['words']:
#        temp_list.append(word)


#documents.append(conference['date'])
#corpus[conference['date']] = np.nan

In [ ]:
import pandas as pd
import numpy as np

my_df = pd.DataFrame()
for i in ['a', 'b', 'c']:
    my_df[i] = np.nan
my_df
for col in my_df.columns:
    print(my_df[col])
#print(a)

In [ ]:
import os
import re

conference_paths = os.listdir('../input/toespraken/')
conference_texts = []

for conference in conference_paths:
    with open(os.path.join('../input/toespraken/', conference), "r", encoding='utf-8') as f:
        conference_texts.append({'filename': conference, 'text': f.readlines()})

save_text = 'rutte'
words = set()

cleaned_conferences = []
all_text_rutte, all_text_de_jonge = [], []
for conference in conference_texts:
    text_rutte, text_de_jonge = [], []
    for line in conference['text'][3:]:
        if save_text == 'rutte':
            text_rutte.append(line)
        if save_text == 'de jonge':
            text_de_jonge.append(line)

        if line.isupper():
            if 'RUT' in line:
                save_text = 'rutte'
            elif 'DE JONGE' in line:
                save_text = 'de jonge'

            else:
                words.add(line)
                save_text = False

    conference_date = conference['filename'][0:-4]

    all_text_rutte.append({'date': conference_date, 'text': text_rutte})
    all_text_de_jonge.append({'date': conference_date, 'text': text_de_jonge})

# Preprocessing

In [ ]:
!py -m spacy download nl_core_news_sm

In [ ]:
import os

import spacy

nlp = spacy.load("nl_core_news_sm")

CONFERENCE_OUTPUT_FOLDER = '../input/conferences/'

In [ ]:
def _preprocess_conference_data(conference_data: list) -> tuple:
    """
    Preprocesses the text of a single conference

    :return: a tuple containing Rutte texts and De Jonge texts respectively per press conference
    """

    # Starts with saving text for Rutte
    save_text = 'rutte'
    words = set()

    # Only keep the sentences of Rutte and De Jonge
    text_rutte, text_de_jonge = [], []
    for line in conference_data:
        if line.isupper():
            if 'RUT' in line:
                save_text = 'rutte'
            elif 'DE JONGE' in line:
                save_text = 'de jonge'
            else:
                words.add(line)
                save_text = 'other'
        else:
            if save_text == 'rutte':
                text_rutte.append(line)
            if save_text == 'de jonge':
                text_de_jonge.append(line)

    return text_rutte, text_de_jonge

In [ ]:
def _preprocess_all_conferences() -> tuple:
    """

    Returns the preprocessed conference data

    :return: a tuple containing Rutte texts and De Jonge texts respectively

    """
    conference_paths = os.listdir(CONFERENCE_OUTPUT_FOLDER)
    all_text_rutte, all_text_de_jonge = [], []

    for conf_file_name in conference_paths:
        with open(f"{CONFERENCE_OUTPUT_FOLDER}/{conf_file_name}", "r", encoding='utf-8') as f:
            text_rutte, text_de_jonge = _preprocess_conference_data(f.readlines())
            conf_date = conf_file_name.replace('.txt', '')
            all_text_rutte.append({'date': conf_date, 'text': text_rutte})
            all_text_de_jonge.append({'date': conf_date, 'text': text_de_jonge})

    return all_text_rutte, all_text_de_jonge

In [ ]:
text_by_speaker = _preprocess_all_conferences()
for i, conferences_list in enumerate(text_by_speaker):
    for j, conference in enumerate(conferences_list):
        # print(i, j, conference['date'])
        full_conference_text = ''.join(conference['text']).replace("\n", " ")
        sentences = re.split(r'(?<![A-Z][a-z]\.)(?<=\.|\?)\s(?<!\w\.\w.\s)', full_conference_text)
        text_by_speaker[i][j]['number_of_sentences'] = len(sentences)
        # print(text_by_speaker[i][j]['number_of_sentences'])
print(text_by_speaker)

In [ ]:
import re

texts = _preprocess_all_conferences()
single_conference_text = texts[0][0]['text']
#print(single_conference_text)
full_conference_text = 'xxx'.join(single_conference_text).replace("\n", " ")
#print(full_conference_text)

# works better than the Spacy sentence splitter
sentences = re.split(r'(?<![A-Z][a-z]\.)(?<=\.|\?)\s(?<!\w\.\w.\s)', full_conference_text)
#print(sentences)
#for sentence in sentences:
#    if len(sentence.split(' ')) < 10:
#        print(sentence)

#texts[0][0]['number_of_sentences'] = len(sentences)
print(len(sentences))

# i = 0
processed_text = nlp(full_conference_text)
sentences = [sentence.text for sentence in processed_text.sents]
print(len(sentences))



#processed_text = [text for text in tqdm(nlp.pipe(texts, n_process=-1, disable=["ner", "parser"]), total=len(texts))]
#lemmatized_text = [[word.lemma_.lower() for word in text if word.pos_ in POStags and not word.is_punct and not word.is_stop] for text in processed_text]
#regexed_text = [[re.sub(r'\W+', '', word) for word in text] for text in lemmatized_text]

In [ ]:
tup = [('a', 1), ('b', 2)]
for i, data in enumerate(tup):
    print(i, data)

b = (1, 3)
c, d = b
c
l = tuple(list(tup))
for i, data in enumerate(l):
    print(l)
[d, c]
3535
'a' in dict(tup)

In [ ]:
a = nlp(full_conference_text)
type(a)

In [ ]:
# tf-idf

from collections import Counter
import math

doc_number = 29
docs_with = 1

#full_conference_text.lower()

words = [token.lemma_ for token in a if not (token.is_stop or token.is_punct or token.is_space)]
words

word_count = Counter(words)
word_count.most_common(20)
word_count

# Get Sorted List
word_list = list(word_count.items())
word_list.sort(key=lambda item: item[1], reverse=True)
print(word_list[:10])

if 'omtt' in dict(word_list):
    print('yes')

# length of word_list
total_uniques = len(word_list)

word_frequency = list()
for i in range(len(word_list)):
    word, count = word_list[i]
    count = count / total_uniques * math.log(doc_number / docs_with)
    word_frequency.append((word, count))
word_frequency